# **Uncertainty Project -- Deep Learning**

---

_Fabio TOCCO, Antoine GUIDON, Yelman YAHI, Anis OUEDGHIRI, Ram NADER_


# Imports


In [ ]:
import os
import tools
from typing import Literal

import torch
import torch.nn as nn
from torch.utils.data import random_split

import torchvision.datasets as datasets

# Setup


In [ ]:
DATA_ROOT = os.path.join(os.path.pardir, "data")
MODELS_ROOT = os.path.join(os.path.pardir, "models")

# Create the directories if they do not exist
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(MODELS_ROOT, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Selected device: {DEVICE}")

# Hyperparameters (DO NOT CHANGE)


In [ ]:
EPOCHS: int = 20
CRITERION: nn.Module = nn.CrossEntropyLoss()
LEARNING_RATE: float = 1e-4
WEIGHT_DECAY: float = 1e-4
BATCH_SIZE: int = 1024

NUM_WORKERS: int = (os.cpu_count() or 0) // 2
print(f"NUM_WORKERS: {NUM_WORKERS}")

# Parameters (change for different training)


In [ ]:
RESIZE_VALUE: int = 32
NORMALIZATION: Literal["MNIST", "ImageNet"] = "MNIST"
SEED: int = 0

tools.seed_everything(seed=SEED)

# Datasets


In [ ]:
data_transforms = tools.get_data_transforms(
    data_root=DATA_ROOT, resize_value=RESIZE_VALUE, normalization=NORMALIZATION
)

train_data = datasets.MNIST(
    DATA_ROOT,
    train=True,
    download=True,
    transform=data_transforms,
)
print(f"Number of train samples: {len(train_data)}")

test_data = datasets.MNIST(
    DATA_ROOT,
    train=False,
    download=True,
    transform=data_transforms,
)
print(f"Number of test samples: {len(test_data)}")

num_classes: int = len(train_data.classes)

# Define the validation set by splitting the training data into 2 subsets (80% training and 20% validation)
n_train_samples = int(len(train_data) * 0.8)
n_validation_samples = len(train_data) - n_train_samples
train_data, validation_data = random_split(
    train_data, [n_train_samples, n_validation_samples]
)

# Experience #1


## DataLoaders


In [ ]:
SHUFFLE: bool = False
train_loader, validation_loader, test_loader = tools.get_loaders(
    train_data,
    validation_data,
    test_data,
    shuffle=SHUFFLE,
    batch_size=BATCH_SIZE,
    drop_last=True,
    num_workers=NUM_WORKERS,
)

## Version 1 - Random weights


### Pre-Training


In [ ]:
PRETRAINED: bool = False
model = tools.make_resnet18(num_classes, pretrained=PRETRAINED)

OPTIMIZER = torch.optim.Adam(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

model_name = tools.get_model_name(
    pretrained=PRETRAINED, shuffle=SHUFFLE, seed=SEED, normalization="MNIST"
)
model_dir = os.path.join(MODELS_ROOT, model_name)
os.makedirs(model_dir, exist_ok=True)

config = {
    "model": "resnet18",
    "pretrained": PRETRAINED,
    "shuffle": SHUFFLE,
    "seed": SEED,
    "normalization": NORMALIZATION,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "weight_decay": WEIGHT_DECAY,
    "optimizer": "Adam",
    "criterion": "CrossEntropyLoss",
    "num_train_samples": len(train_data),
    "num_val_samples": len(validation_data),
    "num_test_samples": len(test_data),
}

### Training loop


In [ ]:
model, train_losses, validation_losses, validation_accuracies = tools.train_model(
    model=model,
    train_loader=train_loader,
    validation_loader=validation_loader,
    criterion=CRITERION,
    optimizer=OPTIMIZER,
    epochs=EPOCHS,
    device=DEVICE,
    file_path=os.path.join(model_dir, model_name + ".pt"),
    verbose=True,
    save_plots=True,
    config=config,
)

### Visualize results


In [ ]:
model = tools.load_model(
    model, os.path.join(model_dir, model_name + ".pt"), device=DEVICE
)

test_loss, test_accuracy = tools.evaluate(
    model, test_loader, criterion=CRITERION, device=DEVICE
)
print(
    f"{model_name} -- Loss on test set: {test_loss:.4f} | Accuracy on test set: {100 * test_accuracy:.2f}%",
)

tools.visualize_predictions(
    model=model, dataset=test_data, device=DEVICE, num_samples=20
)

## Version 2 - Pre-trained weights on ImageNet


### Pre-Training


In [ ]:
PRETRAINED: bool = True
model = tools.make_resnet18(num_classes, pretrained=PRETRAINED)

OPTIMIZER = torch.optim.Adam(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

model_name = tools.get_model_name(
    pretrained=PRETRAINED, shuffle=SHUFFLE, seed=SEED, normalization="MNIST"
)
model_dir = os.path.join(MODELS_ROOT, model_name)
os.makedirs(model_dir, exist_ok=True)

config = {
    "model": "resnet18",
    "pretrained": PRETRAINED,
    "shuffle": SHUFFLE,
    "seed": SEED,
    "normalization": NORMALIZATION,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "weight_decay": WEIGHT_DECAY,
    "optimizer": "Adam",
    "criterion": "CrossEntropyLoss",
    "num_train_samples": len(train_data),
    "num_val_samples": len(validation_data),
    "num_test_samples": len(test_data),
}

### Training loop


In [ ]:
model, train_losses, validation_losses, validation_accuracies = tools.train_model(
    model=model,
    train_loader=train_loader,
    validation_loader=validation_loader,
    criterion=CRITERION,
    optimizer=OPTIMIZER,
    epochs=EPOCHS,
    device=DEVICE,
    file_path=os.path.join(model_dir, model_name + ".pt"),
    verbose=True,
    save_plots=True,
    config=config,
)

### Visualize results


In [ ]:
model = tools.load_model(
    model, os.path.join(model_dir, model_name + ".pt"), device=DEVICE
)

test_loss, test_accuracy = tools.evaluate(
    model, test_loader, criterion=CRITERION, device=DEVICE
)
print(
    f"{model_name} -- Loss on test set: {test_loss:.4f} | Accuracy on test set: {100 * test_accuracy:.2f}%",
)

tools.visualize_predictions(
    model=model, dataset=test_data, device=DEVICE, num_samples=20
)

# Experience #2


## DataLoaders


In [ ]:
SHUFFLE: bool = False
train_loader, validation_loader, test_loader = tools.get_loaders(
    train_data,
    validation_data,
    test_data,
    shuffle=SHUFFLE,
    batch_size=BATCH_SIZE,
    drop_last=True,
    num_workers=NUM_WORKERS,
)